In [1]:
from database import *

def ticks_to_kbars(ticks, interval='1Min'):
    
    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [9]:
import pandas as pd
import talib
api = sj.Shioaji()
api.login(
    person_id='N123569782',
    passwd='hons3146',
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)
connection = sqlite3.connect('data.db')

ModuleNotFoundError: No module named 'stockid'

In [12]:
def get_technical_indicator(kbars):
    
    kbars['KD'] = talib.STOCH(df['High'], 
                              df['Low'], 
                              df['Close'], 
                              fastk_period=9,
                              slowk_period=3,
                              slowk_matype=1,
                              slowd_period=3,
                              slowd_matype=1)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=12, slowperiod=26, signalperiod=9)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    return kbars

In [ ]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=1)
        current_price = kbars['close'][ts]
        
        if (
            kbars.iloc[ts - 3:ts]['KD'].min() < 30 and
            kbars.iloc[ts - 1]['macdhist'] < 0 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
            kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
            kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['macdhist'] < 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time == date.replace(hour=13, minute=0, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()